# Imputation of missing data

### Description: 
In all data science projects, the quality and quantity of data is of great importance. Many times there are missing values on the datasets and this can cause a lot of issues on the accuracy on the data science project's results. For that there are many methods that can be used in order to help impute the missing values in the dataset. In this case the dataset is used for a timeseries analysis and different methods that can be used for imputing missing data will be explained and evaluated.

### Methods that will be tested:
* Simple imputer (with different methods)
* KNN imputer

----

-------

## Step 1: Import the necessary libraries and dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import scipy.sparse as sp
from sklearn.impute import SimpleImputer
from matplotlib import pyplot
from datetime import date, datetime, timedelta

In [2]:
data=pd.read_csv('data/dataset.csv')

## Step 2: Find if there are missing values in the date range you have:
 1. missing dates
 2. NaN values in any other column
 
##### If there are not missing values return the original dataframe

In [3]:
def filtering_data(data):
    final_data=pd.DataFrame()

    if len(data)>1:
        processed_data=find_missing_values(data)
        if processed_data.empty is False:
            print('The result of the Step 2: find_missing_values function is')
            display(processed_data)
            imputed_data=data_imputation(processed_data)
        else:
            imputed_data=data
    else:
        imputed_data=data
    final_data=final_data.append(imputed_data)
    print('The result of the Step 4 and 5 is:')
    display(final_data)
                     
    return final_data      

In [4]:
def find_missing_values(data):
    # this function finds if there are missing dates in the dataframe between the range of the dates that are given 
    # OR other missing values than dates, and returns a dataframe with the missing dates filled and nan values in the rest of the columns
    
    # are there missing dates?
    date_time_str = list(data.date)
    dates=[0]*len(date_time_str)
    for i in range(len(date_time_str)):
        dates[i] = datetime.strptime(date_time_str[i], '%Y-%m-%d')

    start_date = dates[0]
    end_date = dates[len(dates)-1]
    numdays = (end_date - start_date).days
    
    all_dates = []
    for x in range (0, (numdays+1)):
        all_dates.append(start_date + timedelta(days = x))
        
    dates_missing = []
    for i in range (0, len(all_dates)):
        if (all_dates[i] not in dates):
            dates_missing.append(all_dates[i])
        else:
            pass

    d=[0]*len(dates)
    for i in range(len(dates)):
        d[i]=str(dates[i].strftime('%Y-%m-%d'))
    
    if dates_missing != []:
        joined_list = dates+dates_missing # adding up all the dates in the date range of the campaign's stats
        d=[0]*len(joined_list)
        for i in range(len(joined_list)):
            d[i]=str(joined_list[i].strftime('%Y-%m-%d'))
        
        combined_dataframe=pd.DataFrame()
        combined_dataframe['date']=d
        processed_data=data.merge(combined_dataframe,how='right', left_on='date', right_on='date')
        processed_data=processed_data.drop(columns=['index'])
        processed_data=processed_data.sort_values(by=['date'])
        return processed_data
        
        
    # are there NaN values?
    elif data['revenues'].isnull().values.any() or data['spend'].isnull().values.any() or data['budget'].isnull().values.any():
        return data

    return []

## Step 3: Impute the missing values of the dataframe by using different imputations functions

In [5]:
def data_imputation(data):
    # In this function the missing values are filled up by using different imputation methods and 
    # passes the output of all those imputation methods to the evaluation function which finds the best method of imputation for each column 
    # of the dataframe
    
    #KNN imputation
    def knn_imputation(data):
        nan = np.nan
        budget=np.array(data.budget).reshape(-1, 1)
        imputer_b = KNNImputer(n_neighbors=2, weights="uniform")
        new_budget=imputer_b.fit_transform(budget)

        revenues=np.array(data.revenues).reshape(-1, 1)
        imputer_r = KNNImputer(n_neighbors=2, weights="uniform")
        new_revenue=imputer_r.fit_transform(revenues)

        adspend=np.array(data.spend).reshape(-1, 1)
        imputer_a = KNNImputer(n_neighbors=2, weights="uniform")
        new_spend=imputer_a.fit_transform(adspend)

        data['budget']=new_budget
        data['revenues']=new_revenue
        data['spend']=new_spend
        
        return data
        
    
    def mean_imputation(data):
        
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')
        x=np.array(data.budget[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.budget).reshape(-1, 1)
        new_budget=imp.transform(X)
        
        x=np.array(data.revenues[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.revenues).reshape(-1, 1)
        new_revenue=imp.transform(X)
        
        x=np.array(data.spend[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.spend).reshape(-1, 1)
        new_spend=imp.transform(X)
        
        data['budget']=new_budget
        data['revenues']=new_revenue
        data['spend']=new_spend
        
        return data
    
    def median_imputation(data):
        
        imp = SimpleImputer(missing_values=np.nan, strategy='median')
        x=np.array(data.budget[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.budget).reshape(-1, 1)
        new_budget=imp.transform(X)
        
        x=np.array(data.revenues[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.revenues).reshape(-1, 1)
        new_revenue=imp.transform(X)
        
        x=np.array(data.spend[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.spend).reshape(-1, 1)
        new_spend=imp.transform(X)
        
        data['budget']=new_budget
        data['revenues']=new_revenue
        data['spend']=new_spend
        
        return data
    
    def mostfrequent_imputation(data):
        
        imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        x=np.array(data.budget[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.budget).reshape(-1, 1)
        new_budget=imp.transform(X)
        
        x=np.array(data.revenues[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.revenues).reshape(-1, 1)
        new_revenue=imp.transform(X)
        
        x=np.array(data.spend[:20]).reshape(-1, 1)
        imp.fit(x)
        SimpleImputer()
        X = np.array(data.spend).reshape(-1, 1)
        new_spend=imp.transform(X)
        
        data['budget']=new_budget
        data['revenues']=new_revenue
        data['spend']=new_spend
        
        return data
    
    data_knn = knn_imputation(data)
    data_mean = mean_imputation(data)
    data_median = median_imputation(data)
    data_freq = mostfrequent_imputation(data)
    return evaluation_of_methods(data_knn, data_mean, data_median, data_freq)

## Step 4: Evaluate the different imputation functions and return the best for your case

In [6]:
def evaluation_of_methods(data_knn, data_mean, data_median, data_freq):
       
        data=data_knn.copy()
        revenues=data_knn.revenues
        spend=data_knn.spend
        budget=data_knn.budget
        
        knn_r=data_knn.revenues.mean()+data_knn.revenues.std()
        mean_r=data_mean.revenues.mean()+data_mean.revenues.std()
        median_r=data_median.revenues.mean()+data_median.revenues.std()
        freq_r=data_freq.revenues.mean()+data_freq.revenues.std()
        
        knn_s=data_knn.spend.mean()+data_knn.spend.std()
        mean_s=data_mean.spend.mean()+data_mean.spend.std()
        median_s=data_median.spend.mean()+data_median.spend.std()
        freq_s=data_freq.spend.mean()+data_freq.spend.std()
        
        knn_b=data_knn.budget.mean()+data_knn.budget.std()
        mean_b=data_mean.budget.mean()+data_mean.budget.std()
        median_b=data_median.budget.mean()+data_median.budget.std()
        freq_b=data_freq.budget.mean()+data_freq.budget.std()
        
        revenue_list=[knn_r,mean_r,median_r,freq_r]
        spend_list=[knn_s,mean_s,median_s,freq_s]
        budget_list=[knn_b,mean_b,median_b,freq_b]
        
        best_method_r=min(revenue_list)
        best_method_s=min(spend_list)
        best_method_b=min(budget_list)
        
        
        revenue_index = revenue_list.index(best_method_r)
        spend_index = spend_list.index(best_method_s)
        budget_index = budget_list.index(best_method_b)
        
        revenues=[data_knn.revenues,data_mean.revenues,data_median.revenues,data_freq.revenues]
        spend=[data_knn.spend,data_mean.spend,data_median.spend,data_freq.spend ]
        budget=[data_knn.budget,data_mean.budget,data_median.budget,data_freq.budget]
        
        #impute the missing values with the ones that are selected
        data.revenues=revenues[revenue_index]
        data.spend=spend[spend_index]
        data.budget=budget[budget_index]
        return data

In [7]:
final_data=filtering_data(data)

The result of the Step 2: find_missing_values function is


,date,budget,revenues,spend
0,2021-03-01,125.0,200.0,120.0
1,2021-03-02,125.0,372.0,89.0
2,2021-03-03,130.0,290.0,110.0
3,2021-03-04,200.0,250.0,204.0
4,2021-03-05,180.0,200.0,167.0
5,2021-03-06,240.0,290.0,220.0
6,2021-03-07,120.0,150.0,113.0
30,2021-03-08,NaN,NaN,NaN
7,2021-03-09,120.0,192.0,117.0
8,2021-03-10,145.0,100.0,132.0


The result of the Step 4 and 5 is:


,date,budget,revenues,spend
0,2021-03-01,125.000000,200.000000,120.000000
1,2021-03-02,125.000000,372.000000,89.000000
2,2021-03-03,130.000000,290.000000,110.000000
3,2021-03-04,200.000000,250.000000,204.000000
4,2021-03-05,180.000000,200.000000,167.000000
5,2021-03-06,240.000000,290.000000,220.000000
6,2021-03-07,120.000000,150.000000,113.000000
30,2021-03-08,167.333333,231.733333,153.166667
7,2021-03-09,120.000000,192.000000,117.000000
8,2021-03-10,145.000000,100.000000,132.000000


## Step 5: Compare the two dataframes

In [8]:
data=data.drop(columns=['index']) # in order to compare them we should have exactly the same columns 
df_diff = pd.concat([data,final_data]).drop_duplicates(keep=False)
print(f'The difference between the initial data frame and the one after the imputation is:\n\n')
display(df_diff)

The difference between the initial data frame and the one after the imputation is:




,date,budget,revenues,spend
30,2021-03-08,167.333333,231.733333,153.166667
31,2021-03-13,167.333333,231.733333,153.166667
32,2021-03-20,167.333333,231.733333,153.166667
33,2021-03-30,167.333333,231.733333,153.166667
34,2021-03-31,167.333333,231.733333,153.166667
